# Siamese network combined with a variational autoencoder

# Imports

In [ ]:
# # # Credit: https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
from tensorflow import keras
from loaders import TripletSequence
from loaders import PairSequence
import data
import nets

# Globals

In [ ]:
K = keras.backend
codings_size = 10
alpha = 0.5

# Data

In [ ]:
emnist = data.load_dataset('emnist/balanced')
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = emnist.split()

In [ ]:
train_seq = PairSequence(train_x, train_y, samples_per_label=1, batch_size=1024, can_shuffle=True, output='xy')
valid_seq = PairSequence(valid_x, valid_y, samples_per_label=1, batch_size=1024, can_shuffle=True, output='xy')
test_seq  = PairSequence(test_x,  test_y,  samples_per_label=1, batch_size=1024, can_shuffle=True, output='xy')

# Model

In [ ]:
encoder = nets.make_encoder_vae(codings_size)
decoder = nets.make_decoder(codings_size)
snet    = nets.SiameseNetVAE(encoder, decoder)

In [ ]:
siamese_loss = nets.get_contrastive_loss(alpha)
recon_loss   = nets.get_recon_loss()
kld_loss     = nets.get_kld_loss(image_count = 2)

In [ ]:
opt = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True) #0.01

In [ ]:
snet.compile(loss={'recon': recon_loss, 'mean-var': kld_loss, 'distance': siamese_loss}, 
                    metrics={'distance': nets.PairAccuracy(alpha)},
                    loss_weights={'recon': 2.0, 'mean-var': 1.0, 'distance': 3.0},
                    optimizer=opt, 
                    run_eagerly=True
                   )

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=20, min_delta=1/100000, restore_best_weights=True, monitor='val_loss')

In [ ]:
print("Fitting")
snet.fit(train_seq, epochs=500, workers=26, validation_data=(valid_seq), callbacks=[early_stopping])

# Evaluations

In [ ]:
train_triplet_seq = TripletSequence(train_x, train_y, samples_per_label=1, batch_size=1024, can_shuffle=True, is_generator=True)
valid_triplet_seq = TripletSequence(valid_x, valid_y, samples_per_label=1, batch_size=1024, can_shuffle=True, is_generator=True)
test_triplet_seq  = TripletSequence(test_x,  test_y,  samples_per_label=1, batch_size=1024, can_shuffle=True, is_generator=True)

In [ ]:
print("Evaluating training set:")
for i in range(3):
    nets.evaluate_siamese_vae_on_triplets(train_triplet_seq, snet, alpha=alpha)

In [ ]:
print("Evaluating validation set:")
for i in range(3):
    nets.evaluate_siamese_vae_on_triplets(valid_triplet_seq, snet, alpha=alpha)

In [ ]:
print("Evaluating testing set:")
for i in range(3):
    nets.evaluate_siamese_vae_on_triplets(test_triplet_seq, snet, alpha=alpha)

In [ ]:
print("Saving Model")
snet.save("vae_sn_model")